0. когда я открываю файл я выхожу из контекстного менеджера и есть вероятность зарыть его без сохранения (а значит он может или недозаписаться или вообще удалить данные)
0. продумать проверку лога браузера и лога с билетами (сейчас это не оптимально сделано)



Из KLO не строит до пекина

0. Глянуть через фукок

0.продумать запись полетов к api что их нет (потому что если через браузер мы записал, чтобыл запрос, но билетов по факту нет, то к обращаясь к api мы получаем ошибку)
0. генерация из цепочек более сложных перелетов, например из ('KWE', 'PVG'), ('PVG', 'MOW') => ('KWE', 'MOW')
0. сделать стопер, что если цепочка явно дорогая (пока не понятно как это определять), то выкидывать ее сразу из анализу
0. для того чтобы не калбасить все даты, продумать, как отлет или прилет передевать параметром, чтобы не по всем дням проходить
0. в парсер добавить перебор длинных маршрутов
0. очистка ticket base после полной загрузки (удаление дублей)
0. умный перебор: смотреть если перелет из точки А в точку B и в точку C (если нет из B в C то А в B не нужно)


0. когда точек отправления 2 и более сделеать
0. умная фильтрация по столбцам (все одинаковые фичи шли подряд)
0. составление маршрутов когда 2 и более точек
0. при фильтрации добавить duration (когда сравниваю две даты)
0. реализовать систему весов для сортировки
0. киллер фича, это учет часовых поясов аэропортов, даты начала рейса, продолжительности для высчитывания во сколько ты прилетаешь
0. продумать как json дозаписывать
0. сделать нормальную функцию группировки
0. для умной генерации маршрутов нужно навесить кучу ограничений, например, такого не должно быть:

[('KWE', 'TYN'), ('TYN', 'KWE'), ('KWE', 'PVG'), ('PVG', 'MOW')]


In [199]:
#!pip install selenium

In [200]:
# https://support.travelpayouts.com/hc/ru/categories/200358578-API-%D0%B8-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5
# https://support.travelpayouts.com/hc/ru/articles/203956163-Aviasales-API-%D0%B4%D0%BE%D1%81%D1%82%D1%83%D0%BF%D0%B0-%D0%BA-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC-%D0%B4%D0%BB%D1%8F-%D1%83%D1%87%D0%B0%D1%81%D1%82%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2-%D0%BF%D0%B0%D1%80%D1%82%D0%BD%D0%B5%D1%80%D1%81%D0%BA%D0%BE%D0%B9-%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D1%8B#15

In [201]:
ACCESS_TOKEN = 'b8848ff67e92768d2d1977862372d796'

origin = ['KWE']#'KWE'
final_destination = ['TSE']#['LED', 'MOW', 'TLL', 'HEL'] #'LED' 'KLO'
range_date = ['2020-01-12', '2020-01-15']
#number_place = 0

time_sleep = 1
date_start, date_end = range_date[0], range_date[1]

url = 'http://api.travelpayouts.com/v2/prices/month-matrix'

# group = ['DME', 'SVO', 'VKO', 'ZIA']

wishful_point = [] #'KLO'


In [202]:
import requests
import json
import codecs
import os

from itertools import product
from tqdm import tqdm_notebook as tqdm
from collections import Counter

from datetime import datetime, timedelta
from time import sleep
import pandas as pd
from selenium import webdriver

add_zero = lambda x: '0' + str(x) if len(str(x)) < 2 else str(x)
# Используйте тут ваш путь к chromedriver!
chromedriver_path = 'chromedriver.exe'
driver = webdriver.Chrome(executable_path=chromedriver_path) # Этой командой открывается окно Chrome



In [203]:
f = codecs.open('airports.json', encoding='utf-8', mode='r')
data = json.load(f)

In [204]:
cities = list(map(lambda x: x['city_code'], data))
points = list(map(lambda x: x['code'], data))
cities_points = {city:[] for city in cities}
list(map(lambda x, y: cities_points[x].append(y) , cities, points))
cities_points = {k: v for k, v in cities_points.items() if len(v) > 1}
cities_points = list(cities_points.keys())

mapper =  dict(zip(points, cities))
mapper = {k: v for k, v in mapper.items() if v in cities_points}


In [205]:
url_routes = 'http://api.travelpayouts.com/data/routes.json'
routes = json.loads(requests.get(url_routes).text)
route_chain = list(map(lambda x: [x['departure_airport_iata'], x['arrival_airport_iata']], routes))

def group_airport(route, mapper):
    if route[1] in mapper.keys(): route[1] = mapper[route[1]]
    if route[0] in mapper.keys(): route[0] = mapper[route[0]]
    return route



In [206]:
route_chain = list(map(lambda x: group_airport(x, mapper), route_chain))
route_chain = list(set(map(lambda x: tuple(x), route_chain)))

In [207]:
zone = ['Asia/Aden',
 'Asia/Almaty',
 'Asia/Amman',
 'Asia/Anadyr',
 'Asia/Aqtau',
 'Asia/Aqtobe',
 'Asia/Ashgabat',
 'Asia/Baghdad',
 'Asia/Bahrain',
 'Asia/Baku',
 'Asia/Bangkok',
 'Asia/Beirut',
 'Asia/Bishkek',
 'Asia/Brunei',
 'Asia/Chita',
 'Asia/Choibalsan',
 'Asia/Chongqing',
 'Asia/Colombo',
 'Asia/Damascus',
 'Asia/Dhaka',
 'Asia/Dili',
 'Asia/Dubai',
 'Asia/Dushanbe',
 'Asia/Gaza',
 'Asia/Ho_Chi_Minh',
 'Asia/Hong_Kong',
 'Asia/Hovd',
 'Asia/Irkutsk',
 'Asia/Istanbul',
 'Asia/Jakarta',
 'Asia/Jayapura',
 'Asia/Jerusalem',
 'Asia/Kabul',
 'Asia/Kamchatka',
 'Asia/Karachi',
 'Asia/Kathmandu',
 'Asia/Kolkata',
 'Asia/Krasnoyarsk',
 'Asia/Kuala_Lumpur',
 'Asia/Kuching',
 'Asia/Kuwait',
 'Asia/Macau',
 'Asia/Magadan',
 'Asia/Makassar',
 'Asia/Manila',
 'Asia/Muscat',
 'Asia/Nicosia',
 'Asia/Novokuznetsk',
 'Asia/Novosibirsk',
 'Asia/Omsk',
 'Asia/Oral',
 'Asia/Phnom_Penh',
 'Asia/Pontianak',
 'Asia/Pyongyang',
 'Asia/Qatar',
 'Asia/Qyzylorda',
 'Asia/Rangoon',
 'Asia/Riyadh',
 'Asia/Sakhalin',
 'Asia/Samarkand',
 'Asia/Seoul',
 'Asia/Shanghai',
 'Asia/Singapore',
 'Asia/Srednekolymsk',
 'Asia/Taipei',
 'Asia/Tashkent',
 'Asia/Tbilisi',
 'Asia/Tehran',
 'Asia/Thimphu',
 'Asia/Tokyo',
 'Asia/Ulaanbaatar',
 'Asia/Ust-Nera',
 'Asia/Vientiane',
 'Asia/Vladivostok',
 'Asia/Yakutsk',
 'Asia/Yekaterinburg',
 'Asia/Yerevan',
'Indian/Antananarivo',
 'Indian/Christmas',
 'Indian/Cocos',
 'Indian/Comoro',
 'Indian/Mahe',
 'Indian/Maldives',
 'Indian/Mauritius',
 'Indian/Mayotte',
 'Indian/Reunion',
 'Pacific/Apia',
 'Pacific/Auckland',
 'Pacific/Chatham',
 'Pacific/Chuuk',
 'Pacific/Easter',
 'Pacific/Efate',
 'Pacific/Enderbury',
 'Pacific/Fiji',
 'Pacific/Funafuti',
 'Pacific/Galapagos',
 'Pacific/Gambier',
 'Pacific/Guadalcanal',
 'Pacific/Guam',
 'Pacific/Honolulu',
 'Pacific/Johnston',
 'Pacific/Kiritimati',
 'Pacific/Kosrae',
 'Pacific/Kwajalein',
 'Pacific/Majuro',
 'Pacific/Marquesas',
 'Pacific/Midway',
 'Pacific/Nauru',
 'Pacific/Niue',
 'Pacific/Norfolk',
 'Pacific/Noumea',
 'Pacific/Pago_Pago',
 'Pacific/Palau',
 'Pacific/Pohnpei',
 'Pacific/Port_Moresby',
 'Pacific/Rarotonga',
 'Pacific/Saipan',
 'Pacific/Tahiti',
 'Pacific/Tarawa',
 'Pacific/Tongatapu',
 'Pacific/Wake',
 'Pacific/Wallis']

directions = list(filter(lambda x: x['time_zone'] in zone, data))
#directions = list(filter(lambda x: x['country_code'] in ['CN'], directions))
directions = list(map(lambda x: x['city_code'], directions))


In [208]:
def get_next_chain(arrivals, route_chain, final_destination, final = False):
    if final:
        next_chain = filter(lambda x: (x[0] in arrivals) & (x[1] in final_destination), route_chain)
    else:
        next_chain = filter(lambda x: x[0] in arrivals, route_chain)
    return next_chain

def get_arrivals(required_routes):
    if type(required_routes[0][0]) == str:
        arrivals = list(map(lambda x: x[1], required_routes))
    else: 
        arrivals = list(set(map(lambda x: x[-1][1], required_routes)))
    return arrivals

def get_product(required_routes, next_chain):
    if type(required_routes[0][0]) == str:
        required_routes = list(filter(lambda x: x[-2][1] == x[-1][0], product(required_routes, next_chain)))
    else: 
        required_routes = list(filter(lambda x: x[-2][-1][1] == x[-1][0], product(required_routes, next_chain)))
    return required_routes

def get_routes(origin, final_destination, route_chain, number_flights = 4):
    if number_flights == 1:
        required_routes = list(set(filter(lambda x: (x[0] in origin) & (x[1] in final_destination), route_chain)))
    else:
        required_routes = list(set(filter(lambda x: x[0] in origin, route_chain)))
        for n in range(1, number_flights):
            arrivals = get_arrivals(required_routes)
            if n + 1 == number_flights:
                next_chain = get_next_chain(arrivals, route_chain, final_destination, True)
            else:    
                next_chain = get_next_chain(arrivals, route_chain, final_destination)
            required_routes = get_product(required_routes, next_chain)           
    return required_routes

def delete_chains(final_destination, rout):
    s = 1
    for chain in rout:
        if chain[-1] not in final_destination:
            s = s + 1
        else:
            break
    rout = rout[:s]
    return rout
            
def flat_route(route):
    root = route
    result = []
    while type(root[-2]) != str:
        result.append(root[-1])
        root = root[-2]
        
    result.append(root)
    result.reverse()
    return result

In [209]:
def get_wishful_routes(route, wishful_point):
    result = False
    if len(wishful_point) != 0:
        for chain in route:
            if chain[1] in wishful_point or chain[0] in wishful_point:
                result = True
        return result
    else: return True
    
    
def clean_routes(route):
    c = dict(Counter([point for chain in route for point in chain]))
    if 3 in list(c.values()): return False
    else: return True

In [210]:
required_routes = get_routes(origin, final_destination, route_chain, number_flights = 1)
required_routes = list(map(flat_route, required_routes))
required_routes = list(map(lambda x: delete_chains(final_destination, x), required_routes))
required_routes = list(filter(lambda x: get_wishful_routes(x, wishful_point), required_routes))
required_routes = list(filter(lambda x: clean_routes(x), required_routes))

In [211]:
origin = directions
required_routes = list(product(directions, final_destination))

required_routes = list(map(lambda x: [x], required_routes))

In [212]:
all_points = list(set([point for route in required_routes for chain in route for point in chain]))

In [213]:
def list_dates(datetime_start, datetime_end):
    delta = timedelta(days = 1)
    range_dates = []
    date = datetime_start
    while date <= datetime_end:
        range_dates.append(date)
        date = date + delta   
    return range_dates

def open_json(path):
    try:
        with open(path, encoding='utf-8', mode = 'r') as file:
            json_base = json.load(file)
    except FileNotFoundError:
        json_base = []
    return json_base

def write_json(path, json_base):
    with open(path, encoding='utf-8', mode = 'w') as file:
        json.dump(json_base, file)  

def write_file(path, json_base, columns): 
    with open(path, encoding='utf-8', mode = 'a') as file:
        for item in json_base:
            line_file = []
            for key in columns:
                line_file.append(str(item[key]))
            file.write(",".join(line_file) + "\n")

In [214]:
def get_destinations(routes, origin):
    destination_list = list(filter(lambda x: x['departure_airport_iata'] == origin, routes))
    destination_list = list(set(map(lambda x: x['arrival_airport_iata'],destination_list)))
    return destination_list

In [215]:
def update_browser_query(path, days = 5):
    browser_query = open_json(path)
    current_time = datetime.utcnow() - timedelta(days = days)
    browser_query = list(filter(lambda x: datetime.strptime(x['timestamp'], 
                                                            '%Y-%m-%d %H:%M:%S') > current_time, browser_query))  
    write_json(path, browser_query)  

def check_browser_query(date, origin, destination): 
    rewrite = True
    browser_query = open_json('browser_query.json')
    checker = list(filter(lambda x: ((x['date'] == date) & (x['origin'] == origin) & 
                                     (x['destination'] == destination)), browser_query))    
    if len(checker) != 0: rewrite = False          
    return rewrite   

def add_browser_query(date, origin, destination): 
    browser_query = open_json('browser_query.json')
    browser_query.append({'date':date, 'origin':origin, 
         'destination':destination, 'timestamp':datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')})
    write_json('browser_query.json', browser_query)


In [216]:
def update_price_base(path, days = 10):
    columns = ['actual', 'depart_date', 'destination', 'distance', 'duration', 'found_at', 'gate', 'number_of_changes',
     'origin', 'return_date', 'show_to_affiliates', 'trip_class', 'value']  
    if os.path.exists(path):
        data = pd.read_csv(path, parse_dates = ['found_at'])
        current_time = datetime.utcnow() - timedelta(days = days)
        data = data[data['found_at'] > current_time]
        data.to_csv(path, index = False)
    else:
        with open(path, encoding='utf-8', mode = 'a') as file:
            file.write(",".join(columns) + "\n")
        data = pd.read_csv(path, parse_dates = ['found_at'])
    return data

def check_ticket_price(date, origin, destination, base):
    rewrite = base[(base['depart_date'] == date) 
                     & (base['origin'] == origin) 
                     & (base['destination'] == destination)].empty      
    return rewrite


def add_ticket_price(ticket, columns):    
    write_file('ticket_base.txt', ticket, columns)
    
def upload_ticket(token, chain, date, url, ticket_base):
    ticket = []
    columns = list(ticket_base.columns)
    origin, destination = chain
    if check_ticket_price(date, origin, destination, ticket_base):
        params = {'currency': 'rub', 'show_to_affiliates':'false', 'token' : token, 'one_way':'true',
                'origin': origin, 'destination': destination, 'month':date}
        response = requests.get(url, params=params)
        if 'errors' not in json.loads(response.text)['data']:
            ticket = json.loads(response.text)['data']
            add_ticket_price(ticket, columns)
        else:
            pass


In [217]:
def get_price_routes(datetime_start, datetime_end, required_routes):
    tickets = []
    ticket_base = update_price_base('ticket_base.txt')
    columns = list(ticket_base.columns)
    
    update_browser_query('browser_query.json')
    stack = []
    
    parameters = list(product(required_routes, list_dates(datetime_start, datetime_end)))

    for route, datetime_object in tqdm(parameters, total=len(parameters), desc='send_query'):
        stack.append((route, datetime_object))
        month, day = add_zero(datetime_object.month), add_zero(datetime_object.day)
        date = datetime_object.strftime('%Y-%m-%d')
        for chain in route:
            origin, destination = chain

            url_for_cash = f'https://www.aviasales.ru/search/{origin}{day}{month}{destination}1'
            
            if check_ticket_price(date, origin, destination, ticket_base):
                upload_ticket(ACCESS_TOKEN, chain, date, url, ticket_base)

            if check_browser_query(date, origin, destination) and check_ticket_price(date, origin, destination, ticket_base):
                driver.get(url_for_cash)
                sleep(time_sleep)
                while "Извините, доступ к поиску" in driver.page_source:
                    check_point = datetime.now()
                    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'sleeping 20 minutes')
                    while datetime.now() < check_point + timedelta(minutes = 20):
                        if stack:
                            route2, datetime_object2 = stack.pop(0)
                            date2 = datetime_object2.strftime('%Y-%m-%d')
                            for chain2 in route2:
                                upload_ticket(ACCESS_TOKEN, chain2, date2, url, ticket_base)
                        else: sleep(60)
  
                    driver.get(url_for_cash)

                add_browser_query(date, origin, destination)
                
    
    for route, datetime_object in tqdm(stack, total=len(stack), desc='get_price_from'):
        date = datetime_object.strftime('%Y-%m-%d')
        for chain in route:
            upload_ticket(ACCESS_TOKEN, chain, date, url, ticket_base)

In [218]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km


def distance(rout, data):
    lon1, lat1 = tuple(list(filter(lambda x: (x['city_code'] == rout[0]) 
                               or (x['code'] == rout[0]), data))[0]['coordinates'].values())
    
    lon2, lat2 = tuple(list(filter(lambda x: (x['city_code'] == rout[1]) 
                               or (x['code'] == rout[1]), data))[0]['coordinates'].values())
    
    km = haversine(lon1, lat1, lon2, lat2)
    return km


In [219]:
# distance = list(map(lambda x: sum(map(lambda y: distance(y, data),x)), required_routes))
# required_routes = [x for _,x in sorted(zip(distance,required_routes))]

In [220]:
tickets_list = []

datetime_start = datetime.strptime(date_start, '%Y-%m-%d')
datetime_end = datetime.strptime(date_end, '%Y-%m-%d')

#destination_list = get_destinations(routes, origin)

get_price_routes(datetime_start, datetime_end, required_routes)

# for change in range(number_place):
#     ponts_list =  list(set(map(lambda x: x['destination'], tickets_list[change])))
#     tickets = []
#     for point in tqdm(ponts_list, desc=f'for_{change + 1}_point_destination'):
#         if change + 1 < number_place: destination_list = get_destinations(routes, point) + final_destination
#         else: destination_list = final_destination
#         tickets = tickets + get_price_routes(datetime_start, datetime_end, point, destination_list)    
            
#     tickets_list.append(tickets)  


2019-11-01 23:43:41 sleeping 20 minutes
2019-11-02 00:36:17 sleeping 20 minutes
2019-11-02 01:10:58 sleeping 20 minutes
2019-11-02 01:31:29 sleeping 20 minutes


In [221]:
!git add ticket_base.json
!git add browser_query.json
!git commit -m "update jsons"
!git push origin master

[master b80de3c] update jsons
 1 file changed, 1 insertion(+), 1 deletion(-)


To https://github.com/chylkov/aviasales_parser.git
   65c8f83..b80de3c  master -> master


### Нужно порабатать с кодом

In [222]:
df = pd.read_json('ticket_base.json')
df = df[(df['depart_date'] >= date_start) & (df['depart_date'] <= date_end)]

feature = list(df.columns).remove('found_at')

df.sort_values(by = 'found_at', ascending=False, inplace=True)
df.drop_duplicates(inplace = True, keep = 'first', subset = feature)

df.to_csv('routes_price.txt', index = False)

In [223]:
route_feature = ['depart_date', 'origin', 'destination', 'distance', 
                 'duration', 'number_of_changes', 'value']

df_route = df[df['origin'] == origin][route_feature]

for change in range(number_place):
    
    if change == 0:
        df_route = df_route.merge(df[route_feature], 
                                           how = 'left', left_on = 'destination', 
                                           right_on = 'origin', suffixes = ['_from', ''])
        
        df_route.rename(columns={x:x + '_' + str(change + 1) for x in route_feature}, inplace = True)

        df_route = df_route[df_route['depart_date_from'] <= df_route['depart_date_' + str(change + 1)]]
    else:
        df_route = df_route.merge(df[route_feature], 
                                           how = 'left', left_on = 'destination_' + str(change), 
                                           right_on = 'origin')
        
        df_route.rename(columns={x:x + '_' + str(change + 1) for x in route_feature}, inplace = True)
        df_route = df_route[df_route['depart_date_' + str(change)] <= df_route['depart_date_' + str(change+1)]]


value_list = list(filter(lambda x: x[:5] == 'value', list(df_route.columns)))
df_route['total_value'] =  df_route[value_list].sum(axis = 1)

#df_route = df_route[(df_route['depart_date_from'] >= date_start) & (df_route['depart_date_to'] <= date_end)]

df_route.sort_values(by = ['total_value', 'depart_date_from'] , inplace = True)

df_route = df_route[df_route['destination' + str(change+1)].isin(final_destination)]

df_route.reset_index(drop=True, inplace = True)

ValueError: Lengths must match to compare